In [17]:
pip install transformers

In [ ]:
!git clone -b master https://github.com/charles9n/bert-sklearn
!cd bert-sklearn; pip install .

fatal: destination path 'bert-sklearn' already exists and is not an empty directory.
Processing /content/bert-sklearn
  Preparing metadata (setup.py) ... done
  Created wheel for bert-sklearn: filename=bert_sklearn-0.3.1-py3-none-any.whl size=54223 sha256=64f6397748b284b7981c3c1b0ab6c927009be22593baaeeefd7eb380273fd719
  Stored in directory: /root/.cache/pip/wheels/29/f7/03/c9d257b287e1a5ef7ee4300cbbcd70cfbb13e8b6759da24477
Successfully built bert-sklearn
  Attempting uninstall: bert-sklearn
    Found existing installation: bert-sklearn 0.3.1
    Uninstalling bert-sklearn-0.3.1:
      Successfully uninstalled bert-sklearn-0.3.1


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from tqdm import tqdm
import nltk
from sklearn.svm import LinearSVC
from bert_sklearn import BertClassifier
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix

In [ ]:
# Load the data
pubmed_data = pd.read_csv("annotated_pubmed.csv")
eureka_data = pd.read_csv("annotated_eureka.csv")

In [ ]:
pubmed_data.head()

,sentence,label
0,Levels of cholesterol fractions in patients wi...,0
1,Faster aspart and IAsp were confirmed noninfer...,0
2,Major operative morbidity after minimally inva...,0
3,"The promise of combining risk assessment, comm...",0
4,PPARÎ´ peroxisome proliferator-activated recep...,0


In [ ]:
eureka_data.head()

,sentId,sentence,label
0,20824800-w-lss012611-0-title,Low socioeconomic status increases depression ...,3
1,20824800-w-lss012611-1-sentence,A recent study confirmed that low socioeconomi...,1
2,20824800-w-lss012611-2-sentence,"Statistically significant differences in race,...",1
3,20825768-nch-nns011711-0-title,New national study finds 11 500 emergency depa...,0
4,20825768-nch-nns011711-1-sentence,Known by many as one of the least favorite win...,1


In [ ]:
pubmed_data.head()

,sentence,label
0,levels cholesterol fractions patients jia foun...,0
1,faster aspart iasp confirmed noninferior basal...,0
2,major operative morbidity minimally invasive h...,0
3,promise combining risk assessment communicatio...,0
4,pparî´ peroxisome proliferatoractivated recept...,0


In [ ]:
eureka_data.head()

,sentId,sentence,label
0,20824800-w-lss012611-0-title,low socioeconomic status increases depression ...,3
1,20824800-w-lss012611-1-sentence,recent study confirmed low socioeconomic statu...,1
2,20824800-w-lss012611-2-sentence,statistically significant differences race pub...,1
3,20825768-nch-nns011711-0-title,new national study finds 11 500 emergency depa...,0
4,20825768-nch-nns011711-1-sentence,known many one least favorite wintertime chore...,1


In [ ]:
from sklearn.svm import LinearSVC
# Define the label mapping
label_mapping = {'No relationship': 0, 'Direct causal': 1, 'Conditional causal': 2, 'Correlational': 3}

# SVM classifier

X_pub = pubmed_data['sentence']
y_pub = pubmed_data['label']

X_eureka = eureka_data['sentence']
y_eureka = eureka_data['label']

# Split data for cross-validation
X_train, X_test, y_train, y_test = train_test_split(X_pub, y_pub, test_size=0.2, random_state=42)

# TF-IDF vectorization
vectorizer = TfidfVectorizer()
X_train_pub = vectorizer.fit_transform(X_train)
X_train_eureka = vectorizer.transform(X_eureka)

# Train SVM classifier
clf = LinearSVC(C=1)
clf.fit(X_train_pub, y_train)

# Cross-validation
cv_scores = cross_val_score(clf, X_train_pub, y_train, cv=3)
print(f"SVM Cross-Validation Scores for {cv_scores}")

# Evaluate on the test set
X_test_vec = vectorizer.transform(X_test)
y_pred = clf.predict(X_test_vec)
print(f"SVM Classification Report for \n{classification_report(y_test, y_pred)}")

# Train and evaluate SVM and BERT models for both domains


# Perform error analysis, if needed


# Report top 20 features for SVM in each category (you can modify this for each category)

feature_names = np.array(vectorizer.get_feature_names_out())
for label in range(4):
    top_20_features = feature_names[np.argsort(clf.coef_[label])[::-1][:20]]
    print(f"Top 20 features for Label {label}:")
    print(top_20_features)

# Compare SVM and BERT performance
# You can compare the cross-validation results and test set performance for both algorithms.
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score,precision_score, recall_score, roc_auc_score, confusion_matrix

svm_acc = accuracy_score(y_test, y_pred)
print(f"SVM Accuracy: {svm_acc}")
svm_f1 = f1_score(y_test, y_pred, average='weighted')
print(f"SVM F1-score: {svm_f1}")
svm_precision = precision_score(y_test, y_pred, average='weighted')
print(f"SVM svm_precision: {svm_precision}")
svm_recall = recall_score(y_test, y_pred, average='weighted')
print(f"SVM svm_recall: {svm_recall}")
svm_confusion = confusion_matrix(y_test, y_pred)
print("SVM Confusion Matrix:\n", svm_confusion)

SVM Cross-Validation Scores for [0.74019608 0.70955882 0.71936275]
SVM Classification Report for 
              precision    recall  f1-score   support

           0       0.74      0.85      0.79       281
           1       0.77      0.81      0.79       199
           2       0.64      0.46      0.54        39
           3       0.75      0.44      0.55        94

    accuracy                           0.75       613
   macro avg       0.72      0.64      0.67       613
weighted avg       0.75      0.75      0.74       613

Top 20 features for Label 0:
['needed' 'studies' 'require' 'need' 'research' 'required' 'implications'
 'assessment' 'future' 'necessary' 'focus' 'safety' 'option' 'half'
 'including' 'clinicaltrialsgov' 'requires' 'performed' 'needs'
 'considered']
Top 20 features for Label 1:
['associated' 'related' 'association' 'predictor' 'correlated' 'predict'
 'lower' 'controls' 'higher' 'received' 'correlation' 'increased' 'linked'
 'risk' 'likely' 'predictors' 'differenc

In [ ]:
# options
model = BertClassifier(bert_model='bert-base-uncased',
                       label_list=None,
                       num_mlp_hiddens=500,
                       num_mlp_layers=0,
                       epochs=3,
                       max_seq_length=128,
                       train_batch_size=32,
                       eval_batch_size=8,
                       learning_rate=2e-5,
                       warmup_proportion=0.1,
                       gradient_accumulation_steps=1,
                       fp16=False,
                       loss_scale=0,
                       local_rank=-1,
                       use_cuda=True,
                       random_state=42,
                       validation_fraction=0.1,
                       logfile='bert_sklearn.log')

Building sklearn text classifier...


In [ ]:
model = BertClassifier(epochs=2)
print(model)
model.fit(X_train, y_train)

Building sklearn text classifier...
BertClassifier(epochs=2)
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 2204, validation data size: 244



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training  :   0%|          | 0/69 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/bert_sklearn/model/pytorch_pretrained/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1519.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


In [ ]:
from sklearn.metrics import f1_score
bert_acc = model.score(X_eureka, Y_eureka)
print(bert_acc)

# test f1_score
bert_f1 = f1_score(Y_eureka,bert_predictions, average = None)
bert_f1_macro = f1_score(Y_eureka, bert_predictions, average = 'macro')

print(bert_f1)
print(bert_f1_macro)

In [ ]:
# BERT error analysis
err_cnt = 0
for i in range(0, len(X_eureka)):
  if (Y_eureka[i]==0 and bert_predictions[i]==1):
    print(X_eureka[i])
    err_cnt = err_cnt+1
print("errors:", err_cnt)

In [ ]:
# BERT error analysis
err_cnt = 0
for i in range(0, len(X_eureka)):
  if (Y_eureka[i]==0 and bert_predictions[i]==2):
    print(X_eureka[i])
    err_cnt = err_cnt+1
print("errors:", err_cnt)

In [ ]:
# BERT error analysis
err_cnt = 0
for i in range(0, len(X_eureka)):
  if (Y_eureka[i]==0 and bert_predictions[i]==3):
    print(X_eureka[i])
    err_cnt = err_cnt+1
print("errors:", err_cnt)

In [ ]:
# BERT error analysis
err_cnt = 0
for i in range(0, len(X_eureka)):
  if (Y_eureka[i]==0 and bert_predictions[i]==4):
    print(X_eureka[i])
    err_cnt = err_cnt+1
print("errors:", err_cnt)

In [18]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", device=0)

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


RuntimeError: ignored

In [ ]:
pubmed_sentences = list(pubmed_data['sentence'])
eureka_sentences = list(eureka_data['sentence'])

candidate_labels = ["No relationship","Direct causal", "Conditional causal", "Correlational"]
pubmed_predictions = classifier(pubmed_sentences,candidate_labels)
eureka_predictions = classifier(eureka_sentences,candidate_labels)

In [ ]:
pubmed_predicted_labels = [result['labels'][0] for result in pubmed_predictions]
eureka_predicted_labels = [result['labels'][0] for result in eureka_predictions]

In [ ]:
  pubmed_true_labels = list(pubmed_data['label'])
  eureka_true_labels = list(eureka_data['label'])

In [ ]:
# Define a mapping from integer labels to text labels
from sklearn.metrics import precision_score
label_mapping = {
    0: "No relationship",
    1: "Direct causal",
    2: "Conditional causal",
    3: "Correlational"
}

# Convert integer true labels to text labels
pubmed_true_labels_text = [label_mapping[label] for label in pubmed_true_labels]

# Calculate F1 score, recall, and classification report
pubmed_f1_macro = f1_score(pubmed_true_labels_text, pubmed_predicted_labels, average='macro')
pubmed_recall_macro = recall_score(pubmed_true_labels_text, pubmed_predicted_labels, average='macro')
pubmed_precision = precision_score(pubmed_true_labels_text, pubmed_predicted_labels, average='macro')
pubmed_classification_report = classification_report(pubmed_true_labels_text, pubmed_predicted_labels, target_names=candidate_labels)

# Print the results
print("Eureka Macro-F1:", pubmed_f1_macro)
print("Eureka Macro-Recall:", pubmed_recall_macro)
print("Eureka Precision:", pubmed_precision)
print("Classification Report for Pubmed Data:")
print(pubmed_classification_report)

# Define a mapping from integer labels to text labels
label_mapping = {
    0: "No relationship",
    1: "Direct causal",
    2: "Conditional causal",
    3: "Correlational"
}

# Convert integer true labels to text labels
eureka_true_labels_text = [label_mapping[label] for label in eureka_true_labels]

# Calculate F1 score, recall, and classification report
eureka_f1_macro = f1_score(eureka_true_labels_text, eureka_predicted_labels, average='macro')
eureka_recall_macro = recall_score(eureka_true_labels_text, eureka_predicted_labels, average='macro')
eureka_classification_report = classification_report(eureka_true_labels_text, eureka_predicted_labels, target_names=candidate_labels)

# Print the results
print("Eureka Macro-F1:", eureka_f1_macro)
print("Eureka Macro-Recall:", eureka_recall_macro)
print("Classification Report for Eureka Data:")
print(eureka_classification_report)


In [ ]:
# Define a mapping from integer labels to text labels
label_mapping = {
    0: "No relationship",
    1: "Direct causal",
    2: "Conditional causal",
    3: "Correlational"
}

# Convert integer true labels to text labels
eureka_true_labels_text = [label_mapping[label] for label in eureka_true_labels]

# Calculate F1 score, recall, and classification report
eureka_f1_macro = f1_score(eureka_true_labels_text, eureka_predicted_labels, average='macro')
eureka_recall_macro = recall_score(eureka_true_labels_text, eureka_predicted_labels, average='macro')
eureka_classification_report = classification_report(eureka_true_labels_text, eureka_predicted_labels, target_names=candidate_labels)

# Print the results
print("Eureka Macro-F1:", eureka_f1_macro)
print("Eureka Macro-Recall:", eureka_recall_macro)
print("Classification Report for Eureka Data:")
print(eureka_classification_report)


In [19]:
!pip install sentence-transformers
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=cc2657d644d7591a0c0f760ae0f29530ff6606aa412ff92d157fc8347cfb4099
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 15.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_1

In [ ]:
K = 10
tfidf_model = KMeans(n_clusters=K, random_state=0)
tfidf_model.fit(tfidf)
labels=tfidf_model.labels_.tolist()

#  print cluster sizes
def print_cluster_sizes(model):
  cnt_perCluster = {}
  for c in model.labels_:
    cnt_perCluster[c] = cnt_perCluster.get(c, 0)+1
  print(cnt_perCluster)


In [ ]:
# print documents closest to each centroid

import numpy as np
def print_docs_closest_to_centroids(model, vec, n):
  K = len(model.cluster_centers_)
  for j in range(K):
    d = model.transform(vec)[:, j] # transform all docs to cluster-distance space
    idx = np.argsort(d)[:n] # find n docs closest to centroid

    c_idx = [m for m, label in enumerate(model.labels_) if label==j]  # find the index of all docs in cluster j
    print('\n\n======cluster #', j, ', cluster size:', len(c_idx))
    for i in idx:
      if i not in c_idx:
        print('[ this doc is in a different cluster #', model.labels_[i], '>>', corpus[i])
      else:
        print(corpus[i])
  return

In [ ]:
print_docs_closest_to_centroids(tfidf_model, tfidf, 5)

In [ ]:
# Plot the inertia graph
import matplotlib.pyplot as plt
def plot_inertia(inertia_list):
  plt.plot(range(1,len(inertia_list)+1), inertia_list, 'bx-')
  plt.xlabel('Values of K')
  plt.ylabel('Inertia')
  plt.title('The Elbow Method using Inertia')
  plt.show()

In [ ]:
# use the Elbow method to determine the best k
# sometimes it does not work
def elbow(m, corpus):
  inertia_lst = []
  for i in range(1,m+1):
    model = KMeans(n_clusters=i)
    model.fit(corpus)

    inertia=model.inertia_
    inertia_lst.append(inertia)
  plot_inertia(inertia_lst)
elbow(15, tfidf)